### Packages

In [3]:
import os 
import import_ipynb
from time import sleep
# Working Directory
os.chdir("../data")

# Importing notebooks 
from Import_data import *
from Recup_scams_ex import * 

importing Jupyter notebook from Recup_scams_ex.ipynb


# Idées premières

On dispose de données labellisées comme étant des "scams", des arnaques. 
Ce sont essentiellement des adresses ETH, dont il faut regarder le comportement.
Quels features faire sortir de ces adresses, qui caractériseraient les comportements suspects?

Une fois un modèle de caractérisation d'adresses/contrats suspects choisi, on pourra tenter d'associer les adresses entre elles.

## Caractérisation d'une adresse/contrat suspect

1) Un temps d'inactivté, puis un "boom" d'activité peut-être considéré comme un comportement suspect. On part du principe qu'une adresse ETH est en règle générale idle.

2) Un flux de transactions parallèles et/ou de même montant vers une même ou différentes adresses.

3) Un flux de transactions soudain ("établir un rapport, une proportion"?) qui vide le compte d'un seul coup? Eventuellement un outlier sur la proportion quantité d'ETH (in ou out) / temps en h. 

4) Des transactions vers des oeuvres de charité suspicieuses.

5) Une distance vers un exchange assez court, mais pas direct? (Il faut suivre les transactions qui émanent d'une adresse vers une autre ou plusieurs adresses, de même montant ou de montant divisé, qui mènent à un exchange.)

6) On peut s'attendre a des envois d'ETH entre les adresses scams (on peut tenter de compter ces nombres de transactions en communauté) et des petits (ou gros d'ailleurs) envois vers des adresses non suspectes, peut-être des mules (les compter?). Le regroupement de portefeuilles sera traité en seconde part.

7) Eventuellement, regarder les transactions qui entrent dans les exchanges, de montants assez elevées?

8) Peut etre qu'il se passe aussi un grand nombre de transactions "à blanc"? Petits montants, adresses qui semblent pas liées? IL FAUT REGARDER LES GROS MONTANTS. EPS-Voisinage

IL FAUT REGARDER LE DERNIER NOEUD AVANT L EXCHANGE, CE SERA UN PORTEFEUILLE CONTROLE

9) Obfuscation par l'utilisation de token?

PS: Certains gros utilisateurs font du margin trading, prendre cela en compte par rapport à l'activité des adresses
En gros, ils passent de beaucoup d'ETH, a tres peu d'un coup. Ou bien encore quand il n'y a que des transactions qui viennent d'exchanges.

## Temps 

Gestion de la composante temporelle:
On n'a accès qu'aux adresses si les transactions ont lieu.
Il faut aller chercher suffisamment profondément dans les blocks, afin d'avoir un historique large.

De plus, les transactions se suivent, il faut trouver un moyen de prendre ca en compte.

# Feature engineering

In [137]:
def parse_all_ethscam2():

    
    # On importe d'abord l'ensemble des données non formatées du site 
    scam_data = get_ethscam_data()
    
    # Indices des adresses scams uniquement liées a l'ETH
    ind = scam_data.iloc[2] == "ETH" 
    ind = np.argwhere(ind)[:,0]
    
    # Création du tableau "propre"
    
    df = pd.DataFrame()
    
    # Récupération des addresses 
    addresses = np.array(scam_data.iloc[:,ind].columns)
    
    # Récupération des descriptifs 
    prec1 = np.array(scam_data.iloc[1,ind])
    prec2 = np.array(scam_data.iloc[6,ind])
    prec = []
    for i,j in zip(prec1,prec2):
        prec.append(i+" "+j)
    
    # On remplit le tableau 
    df['addresses'] = addresses
    df['precision'] = prec
    
    # On retire les adresses non ETH 
    non_eth_ind = np.argwhere([i[0:2] != '0x' for i in df.addresses])[:,0]
    df = df.drop(non_eth_ind)
    
    # On retire les transactions 
    non_addresses = np.argwhere([len(i) > 42 for i in df.addresses])[:,0]
    df = df.drop(non_addresses)
    
    """
    On complète le tableau avec les balances et les transactions 
    https://api.etherscan.io/api?module=account&action=balancemulti&address={}&tag=latest&apikey=B16PQFQJ9UPYNBP8R6YXF3631EVRTBHUFZ
    EXTRACTION DES BALANCES 
    """
    
    urls = []
    n = 1
    url = str()
    n_iters = np.ceil(df.shape[0]/20) # Le nombre de requetes nécessaires, on ne peut en faire que par batch de 20
    compteur = 0
    
    for add in df.addresses:
        if n%20 != 0:
            url = url + add +',' 
            n += 1
            
            
            
            
        else:
            url = url + add 
            urls.append(url)
            url = str()
            n = 1
            compteur += 1 
            
    if compteur != n_iters:
        url = str()
        for add in df.addresses[(compteur*20):]:
            url = url + add +',' 
            
        url = url[:-1]    
        urls.append(url)
    
    
    balance = []
    for url in urls:
        
        # adresse de l'api
        current_url = "https://api.etherscan.io/api?module=account&action=balancemulti&address={}&tag=latest&apikey=B16PQFQJ9UPYNBP8R6YXF3631EVRTBHUFZ".format(url)
        r = requests.get(url = current_url) 
  
        # extraction du json
        data = r.json() 
    
        res = data['result']
        df_res = pd.DataFrame(data=res)
        df_res['balance'] = df_res['balance'].astype(float)*10**(-18)
        curr_bal = list(df_res['balance'].values)
        balance += curr_bal
        
    df['balance'] = balance
    
    
    """
    EXTRACTION DU NOMBRE DE  TXNS
    NB: IL Y A UN ECART ENTRE LE NB OBTENU SUR ETHERSCAN ET INFURA, ON PREND LA VALEUR ETHERSCAN LORS DE LA FUSION
    """
    API_ENDPOINT = "https://mainnet.infura.io/v3/c6a6f0e057ec465b90b5b85c5cdb5990"
    
    data_list = []
    for j in df.addresses:
        data_list.append({"jsonrpc":"2.0","method":"eth_getTransactionCount","params": [j,"latest"],"id":1})
    
    res_list = []

    for j in data_list:
        data = j
        headers = {"Content-Type": "application/json"}
    
        # sending post request and saving response as response object 
        r = requests.post(url = API_ENDPOINT, json = data, headers=headers) 
  
        # extracting response text  
        pastebin_url = r.text 

        dict_res = eval(pastebin_url)
        res = dict_res['result']
        res = int(res,16)
        res_list.append(res)
        
    
    df['txn'] = res_list
    
    return(df,res_list)
    